# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [20]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [13]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['products', 'customers', 'orders', 'invoices'],
    'definition': [
            'Contains product information with columns: product_id (PK), product_name, price',
            'Contains customer information with columns: customer_id (PK), customer_name, email',
            'Contains order information with columns: order_id (PK), customer_id (FK), product_id (FK), order_date',
            'Contains invoice information with columns: invoice_id (PK), order_id (FK), customer_id (FK) total_amount'
    ]
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0   products  Contains product information with columns: pro...
1  customers  Contains customer information with columns: cu...
2     orders  Contains order information with columns: order...
3   invoices  Contains invoice information with columns: inv...


In [14]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [15]:
print(text_tables)

products: Contains product information with columns: product_id (PK), product_name, price
customers: Contains customer information with columns: customer_id (PK), customer_name, email
orders: Contains order information with columns: order_id (PK), customer_id (FK), product_id (FK), order_date
invoices: Contains invoice information with columns: invoice_id (PK), order_id (FK), total_amount


In [17]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [18]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="show me the history of orders made by customer 123")

In [21]:
print(return_OAI(pqt1))

{
    "tables": ["customers", "orders"]
}


In [24]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Show me top selling products along past year")

In [25]:
print(return_OAI(pqt3))

{
    "tables": ["products", "orders"]
}


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [123]:
def return_OAI(user_message):
    client = OpenAI(api_key=OPENAI_API_KEY)

    context = []
    context.append({"role": "system", "content": user_message})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0,
    )

    return response.choices[0].message.content


In [121]:
import pandas as pd

data = {
    'table': ['products', 'customers', 'orders', 'invoices'],
    'definition': [
        'Contains product information with columns: product_id (PK), product_name, price',
        'Contains customer information with columns: customer_id (PK), customer_name, email',
        'Contains order information with columns: order_id (PK), customer_id (FK), product_id (FK), order_date',
        'Contains invoice information with columns: invoice_id (PK), order_id (FK), customer_id (FK), total_amount'
    ]
}

df = pd.DataFrame(data)


In [124]:
def ask_tables_for_question(question, tables_df):
    text_tables = "\n".join(
        f"{row['table']}: {row['definition']}"
        for _, row in tables_df.iterrows()
    )

    prompt = f"""
Given the following tables and their content definitions,
###Tables
{text_tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""

    return return_OAI(prompt)


print(ask_tables_for_question(
    "show me the history of orders made by customer 123",
    df
))


{
    "tables": ["customers", "orders"]
}
